# Reconstructing the path
In this assignment you will use Spark to compute the shortest path between two vertices. Learn how to compute the distances between a source vertex and all other vertices in a graph. Now, your task is to reconstruct the shortest path, that is a sequence of vertices connected by the edges.

Dataset location: /data/twitter/twitter_sample.txt

Format: user_id \t follower_id



In [1]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local"))

# Increase Java heap memory space

In [2]:
# conf = SparkConf().setAppName("App")
# conf = (conf.setMaster('local[*]')
#         .set('spark.executor.memory', '4G')
#         .set('spark.driver.memory', '45G')
#         .set('spark.driver.maxResultSize', '10G'))
# sc = SparkContext(conf=conf)

In [3]:
#sc._conf.get('spark.driver.memory')


# Path lists

In [17]:
# hadoop = sc._jvm.org.apache.hadoop

# fs = hadoop.fs.FileSystem
# conf = hadoop.conf.Configuration() 
# path = hadoop.fs.Path('/data/twitter')

# for f in fs.get(conf).listStatus(path):
#     print f.getPath()

file:/data/twitter/twitter_sample_small.txt
file:/data/twitter/twitter_sample.txt


In [20]:
parent = dict()
def parse_edge(s):
  user, follower = s.split("\t")
  return (int(user), int(follower))
 
def step(item):
  prev_v, prev_d, next_v = item[0], item[1][0], item[1][1]
  return (next_v, prev_d + 1, prev_v)
 
def complete(item):
  v, old_d, new_d = item[0], item[1][0], item[1][1]
  return (v, old_d if old_d is not None else new_d)
 
n = 400 # number of partitions
edges = sc.textFile("/data/twitter/twitter_sample_small.txt").map(parse_edge).cache()
forward_edges = edges.map(lambda e: (e[1], e[0])).partitionBy(n).persist()
 
x = 12
d = 0
t = 34
path = [t]
parent[x]=-1
distances = sc.parallelize([(x, d)]).partitionBy(n)
found = False
while True:
  candidates = distances.join(forward_edges, n).map(step)
  nodes = candidates.filter(lambda x : x[1]==d+1)
  for node in nodes.toLocalIterator():
    if not parent.has_key(node[0]):
        parent[node[0]] = node[2]
        if(node[0]==t):
            found = True
            break
 
  if not found:
    new_distances = distances.fullOuterJoin(candidates, n).map(complete, True).persist()  
    d += 1
    distances = new_distances.distinct()
  else:
    while(t!=-1):
     t = parent.get(t)
     if t!=-1 :
        path.append(t)
    break
   
 
print(",".join((str(e) for e in path[::-1])))

12,422,53,52,107,20,23,274,34
